In [2]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from pathlib import Path
from collections import OrderedDict
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn import preprocessing
import matplotlib.pyplot as plt
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
import wandb

In [3]:
datafile_path = Path('C:/Users/Ni Ying/Desktop/data/network_datanew.xlsx')
datasets_root = Path('C:/Users/Ni Ying/Desktop/data/working')

In [5]:
raw_dt = pd.read_excel(datafile_path)
raw_dt.head()

,uid,type,betweenness_centrality,degree_centrality,closeness_centrality,pagerank,degree,clustering,ego_betweenness_centrality,ego_pagerank
0,1,pri,0.037270,0.000561,0.242580,0.000120,25,0.016667,0.976667,0.391487
1,12,pri,0.000057,0.000157,0.230282,0.000030,7,0.000000,1.000000,0.469596
2,18,pri,0.002311,0.003682,0.238747,0.000632,164,0.031423,0.804976,0.161289
3,22,pri,0.000003,0.000045,0.198369,0.000013,2,0.000000,1.000000,0.486486
4,35,pri,0.000000,0.000022,0.196286,0.000008,1,0.000000,0.000000,0.500000


In [6]:
data=raw_dt
data.head()

,uid,type,betweenness_centrality,degree_centrality,closeness_centrality,pagerank,degree,clustering,ego_betweenness_centrality,ego_pagerank
0,1,pri,0.037270,0.000561,0.242580,0.000120,25,0.016667,0.976667,0.391487
1,12,pri,0.000057,0.000157,0.230282,0.000030,7,0.000000,1.000000,0.469596
2,18,pri,0.002311,0.003682,0.238747,0.000632,164,0.031423,0.804976,0.161289
3,22,pri,0.000003,0.000045,0.198369,0.000013,2,0.000000,1.000000,0.486486
4,35,pri,0.000000,0.000022,0.196286,0.000008,1,0.000000,0.000000,0.500000


In [7]:
cont_vars = ['betweenness_centrality','degree_centrality','closeness_centrality','pagerank','degree','clustering','ego_betweenness_centrality','ego_pagerank']

In [8]:
test_ratio = 0.3
tr_idx = np.random.choice(np.arange(data.shape[0]), size=round(data.shape[0] * (1 - test_ratio)), replace=False)
tst_idx = list(set(range(data.shape[0])).difference(set(tr_idx)))
len(tr_idx), len(tst_idx)

(44564, 19099)

In [9]:
tr_data = data.iloc[tr_idx]
tst_data = data.iloc[tst_idx]

In [10]:
scaler = preprocessing.StandardScaler().fit(tr_data[cont_vars])

In [11]:
tr_data_scaled = tr_data.copy()
tr_data_scaled[cont_vars] = scaler.transform(tr_data[cont_vars])
tst_data_scaled = tst_data.copy()
tst_data_scaled[cont_vars] = scaler.transform(tst_data[cont_vars])

In [13]:
tst_data, tst_data_scaled

(           uid  type  betweenness_centrality  degree_centrality  \
 0            1   pri                0.037270           0.000561   
 32768  1088810   pri                0.000000           0.000000   
 3           22   pri                0.000003           0.000045   
 4           35   pri                0.000000           0.000022   
 5           37   pri                0.000018           0.000045   
 ...        ...   ...                     ...                ...   
 32748  1088361   pri                0.000000           0.000000   
 32751  1088426   pri                0.000000           0.000000   
 32753  1088458  plat                0.000000           0.000000   
 32756  1088484   pri                0.000000           0.000000   
 32758  1088610   pri                0.000000           0.000000   
 
        closeness_centrality  pagerank  degree  clustering  \
 0                  0.242580  0.000120      25    0.016667   
 32768              0.000000  0.000000       0    0.000000

In [14]:
tr_data_scaled.to_csv(datasets_root/'sn_train.csv', index=False)
tst_data_scaled.to_csv(datasets_root/'sn_test.csv', index=False)